In [3]:
import numpy as np
import math
import scipy.special
inf  = float("inf")


In [64]:
np.zeros(10).shape

(10,)

In [4]:


class Env(object):
    def __init__(self, n_states, n_actions, horizon, discount = 0.99, unique_start = True, deterministic = False):
        self.n_states = n_states
        self.n_actions= n_actions
        self.horizon = horizon
        self.discount = discount
        self.unique_start = unique_start
        self.deterministic = deterministic
        
        if self.unique_start:
            self.start_state = np.random.choice(np.arange(self.n_states))
            self.start_distribution = np.zeros(self.n_states)
            self.start_distribution[self.start_state] = 1
            
        else:
            self.start_distribution = np.random.rand(num_states)
            self.start_distribution /= np.sum(self.start_distribution)
#         self.initial_state_distribution = (1/self.n_states)*np.ones(self.n_states)
        self.mdp_transition_matrices = []
        
        self.get_random_transition_matrices()
        
    def get_random_transition_matrices(self):
        k = self.n_states
        T = np.full(shape = (self.n_states*self.n_actions, self.n_states) , fill_value=0)
        T = np.zeros((self.n_states*self.n_actions, self.n_states))
        
        for i in range(self.n_actions):
            
            Pa = np.identity(k) + \
                    np.random.uniform(low=0., high=.25, size=(k, k))
            Pa /= Pa.sum(axis=1, keepdims=1)
            
            if self.deterministic:
                Pa = np.random.permutation(np.eye(self.n_states))
                
            self.mdp_transition_matrices.append(Pa)
            T[i*k:(i+1)*k,:] = Pa
        
        assert (np.linalg.norm(T.sum(axis = 1)- np.ones(T.shape[0]))) <= 1e-5
        
        self.transition_matrix = T
    
    def generate_trajectories(self):
        # Memoization dictionary to store already computed results
        horizon = self.horizon
        memo_dict = {}

        def generate_recursive(current_time, current_state):
            # Check if the result is already computed
            if (current_time, current_state) in memo_dict:
                return memo_dict[(current_time, current_state)]

            # Base case: when we reach the horizon, return an empty trajectory
            if current_time == horizon:
                return [[]]

            trajectories = [] 

            # Recursive case: try all actions and next states
            for action in range(self.n_actions):
                next_state_probs = self.mdp_transition_matrices[action][current_state] if current_state is not None else self.start_distribution
                
                for next_state in range(self.n_states):
                    # Recursive call
                    if next_state_probs[next_state] != 0:
                        next_trajectories = generate_recursive(current_time + 1, next_state)

                        # Extend the current trajectory with the current action and next state
                        if current_state == None:
                            
                            trajectories.extend([(next_state, action )] + traj for traj in next_trajectories)
                        else:
                            trajectories.extend([(current_state, action, next_state)] + traj for traj in next_trajectories)

            # Memoize the result before returning
            memo_dict[(current_time, current_state)] = trajectories
            return trajectories

        # Start the recursion with time = 0 and no initial state
        # For a unique starting state
        return generate_recursive(0, self.start_state)

In [279]:


def soft_bellman_operation(env, reward):
    
    # Input:    env    :  environment object
    #           reward :  SxA dimensional vector 
    #           horizon:  finite horizon
        
    discount = env.discount
    horizon = env.horizon
    
    if horizon is None or math.isinf(horizon):
        raise ValueError("Only finite horizon environments are supported")
    
    n_states  = env.n_states
    n_actions = env.n_actions
    
    T = env.transition_matrix
    
    V = np.zeros(shape = (horizon, n_states))
    Q = np.zeros(shape = (horizon, n_states,n_actions))
        
    broad_R = reward

    # Base case: final timestep
    # final Q(s,a) is just reward
    Q[horizon - 1, :, :] = broad_R
    # V(s) is always normalising constant
    V[horizon - 1, :] = scipy.special.logsumexp(Q[horizon - 1, :, :], axis=1)

    # Recursive case
    for t in reversed(range(horizon - 1)):
#         next_values_s_a = T @ V[t + 1, :]
#         next_values_s_a = next_values_s_a.reshape(n_states,n_actions)
        for a in range(n_actions):
            Ta = env.mdp_transition_matrices[a]
            next_values_s_a = Ta@V[t + 1, :]
            Q[t, :, a] = broad_R[:,a] + discount * next_values_s_a
            
        V[t, :] = scipy.special.logsumexp(Q[t, :, :], axis=1)

    pi = np.exp(Q - V[:, :, None])

    return V, Q, pi


    

In [293]:
pi

array([[[1.57245616e-05, 9.99984275e-01],
        [5.21088218e-35, 1.00000000e+00]],

       [[1.82596478e-39, 1.00000000e+00],
        [3.09749671e-01, 6.90250329e-01]],

       [[5.01599933e-05, 9.99949840e-01],
        [1.63349336e-35, 1.00000000e+00]],

       [[2.00847295e-43, 1.00000000e+00],
        [9.99754945e-01, 2.45054634e-04]],

       [[5.00000000e-01, 5.00000000e-01],
        [8.19401262e-40, 1.00000000e+00]]])

In [145]:

env.start_distribution

array([1., 0.])

In [333]:
def compute_trajectory_prob(traj, policy, env):
    # the policy is of shape (horizon, n_states, n_actions)
    p = 1.0
    
    for t, (s,a,s_prime) in enumerate(traj):
        
        if t == 0:
            p *= env.start_distribution[s]
        
        # policy probability
        p_policy = policy[t][s][a]
        # tranisition porbability
        p_trans = env.mdp_transition_matrices[a][s][s_prime]
        
        p*= (p_policy*p_trans) 
        
    
    
    return p

def compute_trajectory_probability(traj, pi,env):
    P = []

    
    for t in traj:
        p = compute_trajectory_prob(t,pi, env)
        P.append(p)
        
    # assert that we indeed have a distribution
    assert np.abs(1 - sum(P)) < 1e-4
    
    return P


def discounted_reward_sum(reward,trajectory,env):
    gamma = env.discount
#     gamma = 1
    discounted_reward = 0
    for t, (s,a,sprime) in enumerate(trajectory):
        discounted_reward += (gamma**t)*reward[s][a]
        
    return discounted_reward

In [338]:
for i in range(50):
    n_states = 3
    n_actions = 3
    horizon = 5

    env = Env(n_states,n_actions,horizon, deterministic = False)
    reward = np.zeros(shape = (n_states,n_actions))
    reward[-1,:] = 10
    reward[-1,-1] = 100
    reward = np.random.randn(env.n_states, env.n_actions)
    V, Q, pi = soft_bellman_operation(env,reward )
    traj = env.generate_trajectories()
    l, P, Pz = confirm_boltzman_distribution(reward, traj, env)
#     if not l:
    print(np.linalg.norm(P-Pz))
    print(l)

0.08993390049290516
False
0.07626355140768996
False
0.05310474439222251
False
0.03497576638040493
True
0.036512315481554924
True
0.04277940124418122
True
0.10189595469664141
False
0.04937138482912686
True
0.06121017149002214
False
0.053511098479151684
False
0.05185524697904988
False
0.0289012121031744
True
0.10463514377189267
False
0.11670643981956438
False
0.1353595210320439
False
0.04852164481848946
True
0.17672186490187633
False
0.06293489198513877
False
0.10992820571266598
False
0.11612034834533148
False
0.09588416677706083
False
0.02946626933267505
True
0.2730231529825999
False
0.08932339837048639
False
0.06730740108784096
False
0.027331806733669316
True
0.02491748798198157
True
0.04602119225205987
True
0.13371270116148723
False
0.05079035858006201
False
0.026600797510295492
True
0.05362654348557237
False
0.04217651823971933
True
0.09232144837591005
False
0.17050981974131615
False
0.08941998445095013
False
0.10771125071785499
False
0.08343184977594073
False
0.06861176335879363
Fal

In [340]:
25**5

9765625

In [288]:
reward

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [10., 10., 10.]])

In [324]:
def confirm_boltzman_distribution(reward, traj, env):
    V_1, Q_1, pi = soft_bellman_operation(env,reward)
    
    P = compute_trajectory_probability(traj, pi,env)
 
    P_ziebart = []
    for t in traj:
        rr = discounted_reward_sum(reward,t,env)

        P_ziebart.append(rr)
    
    Z = scipy.special.logsumexp(P_ziebart)
    P_z = np.exp(P_ziebart - Z)
#     print(P_z)
    return True if np.linalg.norm(P- P_z) <= 5e-2 else False, P,P_z

In [255]:
pi

array([[[1.00000000e+00, 1.01122149e-43, 1.01122149e-43],
        [1.01122149e-43, 1.01122149e-43, 1.00000000e+00],
        [8.28596168e-83, 8.28596168e-83, 1.00000000e+00]],

       [[1.00000000e+00, 1.01122149e-43, 1.01122149e-43],
        [1.01122149e-43, 1.01122149e-43, 1.00000000e+00],
        [8.28596168e-83, 8.28596168e-83, 1.00000000e+00]],

       [[1.00000000e+00, 3.00051867e-43, 3.00051867e-43],
        [3.00051867e-43, 3.00051867e-43, 1.00000000e+00],
        [2.45862879e-82, 2.45862879e-82, 1.00000000e+00]],

       [[3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
        [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
        [8.19401262e-40, 8.19401262e-40, 1.00000000e+00]]])

In [281]:
n_states = 3
n_actions = 3
horizon = 4

env = Env(n_states,n_actions,horizon, deterministic = True)
reward = np.zeros(shape = (n_states,n_actions))
reward[-1,:] = 10
reward[-1,-1] = 100
V, Q, pi = soft_bellman_operation(env,reward )
traj = env.generate_trajectories()
l, P, Pz = confirm_boltzman_distribution(reward, traj, env)
print(l)

True


In [314]:
np.unique(np.array(P).round(decimals = 3))

array([0.  , 0.25])

In [315]:
np.unique(np.array(Pz).round(decimals = 3))

array([0.  , 0.25])

In [311]:
max(P)

0.24982517591401415

In [312]:
max(Pz)

0.24979147677944472

In [243]:

    
def check_reward_equivalence(r_1,r_2,T):
    # check weak identifiability equivalence
    diff = []
    for trajectory in T:
        discounted_r_1 = discounted_reward_sum(r_1,trajectory, env)
        discounted_r_2 = discounted_reward_sum(r_2,trajectory, env)
        
        c = discounted_r_1 - discounted_r_2
        diff.append(c)
        
    diff = np.array(diff)
    # need to round for floating-point precision
    
#     print("Unique in d: ",np.unique(diff.round(decimals=4)))

    if np.unique(diff.round(decimals=4)).shape[0] == 1:
#         print("Rewards are equivalent")
        return True
    else:
#         print("Rewards are not equivalent")
        return False
 

# Unit Test for the function above

In [215]:
reward_2 = reward.copy() + 100
# reward_2[0] = 0
# reward_2 = np.zeros((2,2))
d = check_reward_equivalence(reward,reward_2,traj)

Unique in d:  [-679.3465]
Rewards are equivalent


# Code to see if two probability distributions are the same

In [236]:
def check_distribution_equivalence(reward_1, reward_2, env, trajectories):
    
    V_1, Q_1, pi_1 = soft_bellman_operation(env,reward_1)
    V_2, Q_2, pi_2 = soft_bellman_operation(env,reward_2)
    
    P1 = compute_trajectory_probability(trajectories,pi_1,env)
    P2 = compute_trajectory_probability(trajectories,pi_2,env)
    
    return True if np.linalg.norm(np.array(P1)-np.array(P2)) <= 1e-5 else False

In [240]:
check_distribution_equivalence(reward, reward_2, env, traj)

True

# Putting it all together

In [244]:
def check_weak_identifiability(reward_1, reward_2, env, trajectories):
    V_1, Q_1, pi_1 = soft_bellman_operation(env,reward_1)
    V_2, Q_2, pi_2 = soft_bellman_operation(env,reward_2)
    
    P1 = compute_trajectory_probability(trajectories,pi_1,env)
    P2 = compute_trajectory_probability(trajectories,pi_2,env)
    
    # first check if the distributions are unqiue:
    first_branch = True if np.linalg.norm(np.array(P1)-np.array(P2)) <= 1e-5 else False
    
    # if first_branch == False, then we are on the first left branch
    if first_branch == False:
        test_1 = check_reward_equivalence(reward_1,reward_2,trajectories)
        if test_1:
            print("Not weakly identifiable")
            return
        else:
            print("Weakly Identifabiable")
            return
    
    # first_branch == True, so since we reach here, then we are on the right side of the tree
    # check reward equivalence for the equal trajectories
    test_1 = check_reward_equivalence(reward_1,reward_2,trajectories)
    
    if test_1 == False:
        print("Not weakly identifiable")
        return
    
    # now we need to check proper model:
    if test_1 and first_branch:
        print("weakly identifiable")
        return
    

In [256]:
reward_1 = reward.copy()
reward_2 = reward.copy() + 5
reward_2[0] = 0
reward_2[0][1] = 1
check_weak_identifiability(reward, reward_2, env, traj)

Weakly Identifabiable


In [257]:
reward_2

array([[ 0.,  1.],
       [15., 15.]])

In [258]:
reward_1

array([[ 0.,  0.],
       [10., 10.]])

In [259]:
check_distribution_equivalence(reward_1, reward_2, env, traj)

False

In [274]:
np.random.permutation(np.eye(5))

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])